In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import requests
import json
import pprint
import time

import pandas as pd

# fix ssl certificate (needed for MacOS sometimes)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

/Users/davidshaw/Projects/minWage/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
CAP_FRIENDLY_BASE_URL = 'https://www.capfriendly.com'

def get_soup(url):
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [3]:
url = 'https://www.capfriendly.com/retained-salary-transactions'
rs_soup = get_soup(url)

retained_salaries = []
for row in rs_soup.find('table', {'id': 'rst'}).find_all('tr')[1:]:
    row_data = [row.find('td').find('a').get('href')]
    for td in row.find_all('td'):
        row_data.append(td.get_text())
        
    retained_salaries.append(row_data)

In [4]:
df = pd.DataFrame(retained_salaries,
                  columns=['playerLink', 'name', 'originTeam', 'dateOfTrade', 'percentRetained',
                           'originalCapHit', 'retainedCapHit', 'activeContract'])
df

,playerLink,name,originTeam,dateOfTrade,percentRetained,originalCapHit,retainedCapHit,activeContract
0,/players/reilly-smith,Reilly Smith,PIT,"Jul. 1, 2024",25.00%,"$5,000,000","$1,250,000",✔
1,/players/ilya-mikheyev,Ilya Mikheyev,VAN,"Jun. 26, 2024",15.00%,"$4,750,000","$712,500",✔
2,/players/joonas-korpisalo,Joonas Korpisalo,OTT,"Jun. 24, 2024",25.00%,"$4,000,000","$1,000,000",✔
3,/players/jacob-markstrom,Jacob Markström,CGY,"Jun. 19, 2024",31.25%,"$6,000,000","$1,875,000",✔
4,/players/tyler-toffoli,Tyler Toffoli,NJD,"Mar. 8, 2024",50.00%,"$4,250,000","$2,125,000",
...,...,...,...,...,...,...,...,...
204,/players/ben-scrivens,Ben Scrivens,TOR,"Jun. 23, 2013",10.20%,"$612,500","$62,500",
205,/players/matt-frattin,Matt Frattin,TOR,"Jun. 23, 2013",50.00%,"$875,000","$437,500",
206,/players/jussi-jokinen,Jussi Jokinen,CAR,"Apr. 3, 2013",30.00%,"$3,000,000","$900,000",
207,/players/jason-pominville,Jason Pominville,BUF,"Apr. 3, 2013",15.00%,"$5,300,000","$795,000",


In [5]:
df.to_csv('retainedSalaryTransactions_tilJuly2024.csv', index=False)